In [1]:
!apt-get update
!apt-get install -y locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,442 kB]
Get:14 http://archive.ubuntu.com/u

In [3]:
!pip install colab-env -qU
import colab_env

  Preparing metadata (setup.py) ... done


MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install python-dotenv

import os

import dotenv
from google.colab import drive

drive.mount('/content/drive')

dotenv.load_dotenv('/content/drive/MyDrive/.env')

os.environ.get('CUDA_ACCELERATOR')

MessageError: Error: credential propagation was unsuccessful

In [4]:
!python -m cupyx.tools.install_library --cuda 11.x --library cutensor

By downloading and using cuTENSOR, you accept the terms and conditions of the NVIDIA cuTENSOR Software License Agreement:
  https://docs.nvidia.com/cuda/cutensor/license.html

Installing cutensor 1.6.2 for CUDA 11.x to: /root/.cupy/cuda_lib/11.x/cutensor/1.6.2
Traceback (most recent call last):
  File "/usr/lib/python3.10/http/client.py", line 482, in read
    s = self._safe_read(self.length)
  File "/usr/lib/python3.10/http/client.py", line 631, in _safe_read
    data = self.fp.read(amt)
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.10/ssl.py", line 1303, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.10/ssl.py", line 1159, in read
    return self._sslobj.read(len, buffer)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_co

In [5]:
pip install cupy-cuda12x cutensor-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 MB 6.5 MB/s eta 0:00:00


In [26]:
import numpy as np
import cupy as cp
import cutensor as cutensor

cp.cuda.Stream.null.synchronize()

In [ ]:
arr = cp.ones((1000,500,500))

#makes us wait for gpu to finish before returning
cp.cuda.Stream.null.synchronize()

In [55]:
A = cp.array([[[1,2,3],
               [3,4,5],
               [6,7,8]],

              [[9,10,11],
               [12,13,14],
               [15,16,17]]],)

B = cp.random.rand(4,3)


print("A: ", A)
print("B: ", B)
print(A.shape)
print(B.shape)


A:  [[[ 1  2  3]
  [ 3  4  5]
  [ 6  7  8]]

 [[ 9 10 11]
  [12 13 14]
  [15 16 17]]]
B:  [[0.19220529 0.52466003 0.76355666]
 [0.02198596 0.0725163  0.44633282]
 [0.45685477 0.04452992 0.20356587]
 [0.91817161 0.64216566 0.55600169]]
(2, 3, 3)
(4, 3)


In [14]:
result = cp.einsum('ijk,jkl->il', A, B)
print("Result with cp.einsum: ", result)

# ijk the dimensions of A, jkl the dimensions of B, il the dimensions of the einstein sum
# contraction happens over the j and k dimensions
cp.cuda.Stream.null.synchronize()

ValueError: Size of label 'j' for operand 1 (2) does not match previous terms (3).

In [10]:
# using cutensor

#need descriptors to describe shape, data type and memory of tensor
desc_A = cutensor.create_tensor_descriptor(A)

desc_B = cutensor.create_tensor_descriptor(B)

# Empty output tensor of the shape of the einstein sum of dimensions il
output = cp.empty((2,3), dType = A.dtype)

desc_output = cutensor.create_tensor_descriptor(output)


AttributeError: module 'cutensor' has no attribute 'create_tensor_descriptor'

In [ ]:
#create description of contraction of the tensors
contract_desc = cutensor.einsum_expr(('i','j','k'),('j','k','l'),('i','l'))
#('i','j','k') dimensions of A,
#('j','k','l') dimensions of B,
#('i','l') dimensions of output

#scale factor for contraction operation
alpha = 1.0
#scale factor for output
beta = 0.0

#contraction plan
contraction_plan = cutensor.create_contraction_plan(contract_desc, desc_A, desc_B, desc_output)

In [ ]:
#Perform contraction using contraction plan
cutensor.contraction(contraction_plan, alpha, A, desc_A, B, desc_B, beta, output, desc_output)

cp.cuda.Stream.null.synchronise()
print("Result with cutensor: ", output)

In [89]:

#Task 1 function/abstract contraction passing the tensor, matrix, and dimension

def contract(A, X, dim):
    # Takes tensor A of unknown number of dimensions
    # Takes matrix h with 2 dimensions Kk
    # Takes int dim as the dimension to contract over
    if (type(dim) != int and type(dim) == str) :
      raise ValueError(f"Dim must be an int not a string")

    # Get the dimensions of the tensor and matrix
    A_shape = cp.array(A.shape)
    X_shape = cp.array(X.shape)

    # Check that the size of the dimension matches
    if A_shape[dim] != X_shape[1]:
      raise ValueError(f"Dimension mismatch: A has size {A_shape[dim]} along dimension {dim}, "f"but X has size {X_shape[1]}")

    # Compress the dimensions not involved in contraction

    reshape_left = cp.prod(A_shape[:dim]).item()
    contract_dim_size = A_shape[dim].item()
    reshape_right = cp.prod(A_shape[dim+1:]).item()

    #print(f"Reshape Left: {reshape_left} (type: {type(reshape_left)})")
    #print(f"Contract Dim Size: {contract_dim_size} (type: {type(contract_dim_size)})")
    #print(f"Reshape Right: {reshape_right} (type: {type(reshape_right)})")


    As = A.reshape(reshape_left, contract_dim_size, reshape_right)

    result = cp.einsum("jkl, Kk->jKl", As, X)
    #print(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())
    result = result.reshape(*A_shape[:dim].tolist(), X.shape[0], *A_shape[dim + 1:].tolist())

    return result

    #As = A.reshape( np.prod(A.shape[:2]), A.shape[2], np.prod(A.shape[3:]))
    #Ahs = np.einsum("Kk,ikl->iKl",h,As)
    #np.allclose(Ahs.ravel(),Ah.ravel())
#
#
    #In [17]: A = np.random.rand(3,2,4,6,1)

    #In [18]: h = np.random.rand(4,4)

    #In [19]: Ah = np.einsum("Kk,ijklm->ijKlm",h,A)

    #In [20]: As = A.reshape( np.prod(A.shape[:2]), A.shape[2],
    #np.prod(A.shape[3:]))

    #In [21]: Ahs = np.einsum("Kk,ikl->iKl",h,As)

    #In [22]: np.allclose(Ahs.ravel(),Ah.ravel())
    #In [23]: np.allclose(Ahs.reshape(Ah.shape),Ah)
#
    #A = A.reshape(A.prod(A.shape[:]))



In [91]:
A = np.random.rand(2,3,2,4,7,8)
h = np.random.rand(4,7)
Ah = np.einsum("Kk,hijlkm->hijlKm",h,A)

Ahs = contract(A, h, 4)

print(cp.allclose(Ahs.ravel(),Ah.ravel()))
print(len(Ahs.ravel()))
print(len(Ah.ravel()))
print(cp.allclose(Ahs.reshape(Ah.shape),Ah))
print(cp.allclose(Ahs,Ah))

True
1536
1536
True
True
